# Loop 27 Strategic Analysis

## Key Findings:
1. All local search approaches converge to 70.624381
2. Gap to target: 1.705 points (2.41%)
3. Target IS achievable (above theoretical minimum)
4. Need fundamentally different approach

In [1]:
# Analyze per-N scores to find where improvements are most needed
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def parse_s_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(trees):
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def calculate_score(positions, n):
    trees = [create_tree_polygon(p[0], p[1], p[2]) for p in positions]
    side = get_bounding_box_side(trees)
    return (side ** 2) / n

# Load current best
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].apply(parse_s_value)
df['y'] = df['y'].apply(parse_s_value)
df['deg'] = df['deg'].apply(parse_s_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

# Calculate per-N scores
tree = Polygon(TREE_TEMPLATE)
tree_area = tree.area
print(f'Single tree area: {tree_area:.6f}')

scores = {}
for n in range(1, 201):
    group = df[df['n'] == n]
    positions = np.array([[row['x'], row['y'], row['deg']] for _, row in group.iterrows()])
    scores[n] = calculate_score(positions, n)

# Calculate efficiency and improvement potential
print('\n=== Top 20 N values with most improvement potential ===')
efficiency = [(n, scores[n], scores[n] / tree_area, scores[n] - tree_area) for n in range(1, 201)]
efficiency.sort(key=lambda x: -x[3])  # Sort by improvement potential

for n, score, eff, potential in efficiency[:20]:
    print(f'N={n:3d}: score={score:.6f}, efficiency={eff:.3f}x, potential={potential:.6f}')

print(f'\nTotal score: {sum(scores.values()):.6f}')
print(f'Target: 68.919154')
print(f'Gap: {sum(scores.values()) - 68.919154:.6f}')

Single tree area: 0.245625



=== Top 20 N values with most improvement potential ===
N=  1: score=0.661250, efficiency=2.692x, potential=0.415625
N=  2: score=0.450779, efficiency=1.835x, potential=0.205154
N=  3: score=0.434745, efficiency=1.770x, potential=0.189120
N=  5: score=0.416850, efficiency=1.697x, potential=0.171225
N=  4: score=0.416545, efficiency=1.696x, potential=0.170920
N=  7: score=0.399897, efficiency=1.628x, potential=0.154272
N=  6: score=0.399610, efficiency=1.627x, potential=0.153985
N=  9: score=0.387415, efficiency=1.577x, potential=0.141790
N=  8: score=0.385407, efficiency=1.569x, potential=0.139782
N= 15: score=0.376950, efficiency=1.535x, potential=0.131325
N= 10: score=0.376630, efficiency=1.533x, potential=0.131005
N= 21: score=0.376451, efficiency=1.533x, potential=0.130826
N= 20: score=0.376057, efficiency=1.531x, potential=0.130432
N= 22: score=0.375258, efficiency=1.528x, potential=0.129633
N= 11: score=0.374924, efficiency=1.526x, potential=0.129299
N= 16: score=0.374128, effic

In [2]:
# Analyze what percentage of improvement we need per N
total_score = sum(scores.values())
target = 68.919154
gap = total_score - target

print(f'Total gap to close: {gap:.6f}')
print(f'\nIf we improve each N by the same percentage:')
required_pct = gap / total_score * 100
print(f'Required improvement: {required_pct:.2f}%')

print(f'\nIf we focus on N=1-20 (highest potential):')
n1_20_score = sum(scores[n] for n in range(1, 21))
n1_20_potential = sum(scores[n] - tree_area for n in range(1, 21))
print(f'N=1-20 current score: {n1_20_score:.6f}')
print(f'N=1-20 improvement potential: {n1_20_potential:.6f}')
print(f'If we capture 50% of N=1-20 potential: {n1_20_potential * 0.5:.6f} improvement')

print(f'\nIf we focus on N=1-50:')
n1_50_score = sum(scores[n] for n in range(1, 51))
n1_50_potential = sum(scores[n] - tree_area for n in range(1, 51))
print(f'N=1-50 current score: {n1_50_score:.6f}')
print(f'N=1-50 improvement potential: {n1_50_potential:.6f}')
print(f'If we capture 20% of N=1-50 potential: {n1_50_potential * 0.2:.6f} improvement')

Total gap to close: 1.705227

If we improve each N by the same percentage:
Required improvement: 2.41%

If we focus on N=1-20 (highest potential):
N=1-20 current score: 8.053174
N=1-20 improvement potential: 3.140674
If we capture 50% of N=1-20 potential: 1.570337 improvement

If we focus on N=1-50:
N=1-50 current score: 19.033316
N=1-50 improvement potential: 6.752066
If we capture 20% of N=1-50 potential: 1.350413 improvement


In [3]:
# Check what the theoretical best score would be for different efficiency levels
print('=== Theoretical scores at different efficiency levels ===')
for eff_mult in [1.0, 1.1, 1.2, 1.3, 1.4, 1.5]:
    theoretical = tree_area * eff_mult * 200
    print(f'Efficiency {eff_mult:.1f}x: {theoretical:.6f}')

print(f'\nCurrent average efficiency: {total_score / (tree_area * 200):.3f}x')
print(f'Target average efficiency: {target / (tree_area * 200):.3f}x')

=== Theoretical scores at different efficiency levels ===
Efficiency 1.0x: 49.125000
Efficiency 1.1x: 54.037500
Efficiency 1.2x: 58.950000
Efficiency 1.3x: 63.862500
Efficiency 1.4x: 68.775000
Efficiency 1.5x: 73.687500

Current average efficiency: 1.438x
Target average efficiency: 1.403x


In [4]:
# Analyze the distribution of angles in the current solution
print('=== Angle distribution analysis ===')
for n in [1, 2, 3, 4, 5, 10, 20, 50, 100]:
    group = df[df['n'] == n]
    angles = group['deg'].values % 360
    unique_angles = np.unique(np.round(angles, 1))
    print(f'N={n}: angles = {unique_angles[:5]}... ({len(unique_angles)} unique)')

=== Angle distribution analysis ===
N=1: angles = [45.]... (1 unique)
N=2: angles = [ 23.6 203.6]... (2 unique)
N=3: angles = [ 66.4 113.6 155.1]... (3 unique)
N=4: angles = [156.4 336.4]... (2 unique)
N=5: angles = [ 23.6  66.4 112.6 204.3 293.6]... (5 unique)
N=10: angles = [ 21.4  23.6  66.3  70.4 115.6]... (10 unique)
N=20: angles = [20.8 22.8 24.6 33.1 63.7]... (20 unique)
N=50: angles = [13.3 20.7 23.6 25.1 28.1]... (43 unique)
N=100: angles = [65.8 65.9 66.1 66.3 66.6]... (40 unique)
